In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

### Paper definitions:
Enhancer paper = "Enhancer Identification from DNA sequence using Transfer and Adversarial Deep Learning"  
triple GAN = "Triple Generative Adversarial Nets"

### Loading the data

In [ ]:
#TODO

### Scratchpad

In [15]:

kernel_shape=[4,9]
input_channels = 1
num_kernels = 20

weights_shape = kernel_shape + [input_channels, num_kernels]
print(weights_shape)
print([1] + [2, 2] + [1])

[4, 9, 1, 20]
[1, 2, 2, 1]


### Defining helper functions


In [12]:
def conv_layer(name_scope, input_tensor, num_kernels, kernel_shape,
               stride=1, padding="VALID", relu=True, 
               batch_normalize=False, batch_normalize_training=True, 
               name_suffix=None):
    """
    Return a convolution layer, possibly with a ReLU at the end.
    :param name_scope:   where the variables live
    :param input_tensor: of shape [batch, in_height, in_width, in_channels], e.g. [15 500 4 1]
    :param num_kernels:  number of kernels to use for this conv. layer
    :param kernel_shape: the shape of the kernel to use, [height, width]
    """
    name_suffix = name_suffix if name_suffix else ""
    
    #E.g. batch_size x 500x4x1 for the first input
    input_shape = input_tensor.get_shape().as_list()
    input_channels = input_shape[-1]
    
    #not really sure why I'm using the name_scope, I think it's mostly for presentation purposes
    with tf.name_scope(name_scope):
        
        weights_shape = kernel_shape + [input_channels, num_kernels]
        init_vals_weights = tf.truncated_normal(weights_shape, stddev=math.sqrt(2 / float(input_channels)))
        filter_weights = tf.Variable(init_vals_weights, name='weights'+name_suffix)
    
        biases = tf.Variable(tf.zeros([num_kernels]), name='biases'+name_suffix)
        
        #Define a convolutional layer
        layer = tf.nn.conv2d(input_tensor, filter_weights, strides=[1, stride, stride, 1], padding=padding) + biases
        
        #TODO: batch_normalization layer
        
        #Add ReLU if specified
        if relu:
            layer = tf.nn.relu(layer, name="relu_"+name_suffix)
            
        return layer

    
def max_pool_layer(name_scope, input_tensor, pool_size, strides = None, padding="SAME"):
    """
    Return a max pool layer.
    """
    if not strides:
        strides = [1] + pool_size + [1]
       
    #TODO: is name_scope really needed?
    with tf.name_scope(name_scope):
        layer = tf.nn.max_pool(input_tensor, [1] + pool_size + [1], strides=strides, padding=padding)
        return layer


def dropout_layer(name_scope, input_tensor, keep_prob=0.5):
    """
    Return a dropout layer.
    """
    #TODO: is name_scope really needed?
    with tf.name_scope(name_scope)
        return tf.nn.droupout()

In [ ]:
def prepare_classifier(data):
    """
    Return the same classifier, with architecture the same as they used in Enhancer paper.
    """
    # 20 filters, each of size batch x 4x9x1
    # TODO: make a reverse filter conv layer like in the Enhancer paper 
    l1 =  conv_layer(name_scope="layer1", data, num_kernels=20, kernel_shape=[4, 9], relu=True):
    
    

# Note: below is taken from https://github.com/uclaacmai/Generative-Adversarial-Network-Tutorial.git

### Assemble components (discriminator, generator, classifier)

In [ ]:
#TODO: adjust to our needs
sess = tf.Session()
z_dimensions = 100
batch_size = 16
tf.reset_default_graph() #Since we changed our batch size (from 1 to 16), we need to reset our Tensorflow graph

sess = tf.Session()
x_placeholder = tf.placeholder("float", shape = [None,500,4,1]) #Placeholder for input images to the discriminator
z_placeholder = tf.placeholder(tf.float32, [None, z_dimensions]) #Placeholder for input noise vectors to the generator

Dx = discriminator(x_placeholder) #Dx will hold discriminator prediction probabilities for the real MNIST images
Gz = generator(z_placeholder, batch_size, z_dimensions) #Gz holds the generated images
Dg = discriminator(Gz, reuse=True) #Dg will hold discriminator prediction probabilities for generated images


### Specify loss

In [ ]:
#TODO: specify the right losses etc

g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Dg, labels = tf.ones_like(Dg))) # ensure forward compatibility: function needs to have logits and labels args explicitly used

d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Dx, labels = tf.ones_like(Dx)))
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Dg, labels = tf.zeros_like(Dg)))
d_loss = d_loss_real + d_loss_fake


### Specify optimizers

In [ ]:
#TODO: adjust to our needs
tvars = tf.trainable_variables()
d_vars = [var for var in tvars if 'd_' in var.name]
g_vars = [var for var in tvars if 'g_' in var.name]

print(tf.get_variable_scope().reuse)
adam = tf.train.AdamOptimizer()
trainerD = adam.minimize(d_loss, var_list=d_vars)
trainerG = adam.minimize(g_loss, var_list=g_vars)

### Train the stuff

In [ ]:
#TODO: make it compatible with our dataset

sess.run(tf.global_variables_initializer())
iterations = 3000
for i in range(iterations):
    z_batch = np.random.normal(-1, 1, size=[batch_size, z_dimensions])
    real_image_batch = mnist.train.next_batch(batch_size)
    real_image_batch = np.reshape(real_image_batch[0],[batch_size,28,28,1])
    _,dLoss = sess.run([trainerD, d_loss],feed_dict={z_placeholder:z_batch,x_placeholder:real_image_batch}) #Update the discriminator
    _,gLoss = sess.run([trainerG,g_loss],feed_dict={z_placeholder:z_batch}) #Update the generator 